In [ ]:
latest_gameweek = 2

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.graph_objects as go
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 300)
#pd.set_option('display.max_rows', 100)

# Data

In [ ]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
display(projections.head())
display(projections.shape)

In [ ]:
filepath = Path('../data/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='23-24']
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
df = fpl_df.groupby('name').last().reset_index()[['name', 'team_name', 'element_type', 'now_cost', 'points_per_game', 'total_points']]
df

In [ ]:
fpl_df.gameweek_xPoints.isnull().sum()

In [ ]:
fpl_df.groupby('name')['gameweek_xPoints'].mean()

# Analysis